**Group Name:** *Closed AI*

**Members**: 

    1. Malak Mehrez

    2. Sergio Khalil

    3. Tamer AlBouz


**Kindly check the README.md File**   

In [ ]:
# needed librairies
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from keras.callbacks import ModelCheckpoint

**Loading Data**

In [ ]:
# grant access to the google drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

# load the dataset located in a folder named "ML" that's in your root Google Drive
data = pd.read_csv('/content/drive/MyDrive/ML/HIGGS_train.csv', dtype={'8': float, '21': float})

# define the headers of the dataset
column_names = ['class_label', 'lepton_pt', 'lepton_eta', 'lepton_phi', 'missing_energy_magnitude', 'missing_energy_phi',
                'jet_1_pt', 'jet_1_eta', 'jet_1_phi', 'jet_1_btag', 'jet_2_pt', 'jet_2_eta', 'jet_2_phi', 'jet_2_btag',
                'jet_3_pt', 'jet_3_eta', 'jet_3_phi', 'jet_3_btag', 'jet_4_pt', 'jet_4_eta', 'jet_4_phi', 'jet_4_btag',
                'm_jj', 'm_jjj', 'm_lv', 'm_jlv', 'm_bb', 'm_wbb', 'm_wwbb']

# assign the headers to the data
data.columns=column_names

cleaned_data = data.copy();

**Exploring Data**

In [ ]:
# check the first few rows of the data
print("The first few rows:\n")
print(data.head())
print("\n")

# retrive the information of the data
print("Data Information:\n")
print(data.info())
print("\n")

# retrive basic statistics about the data
print("Data Statistics:\n")
print(data.describe())
print("\n")

# retrive the shape of the data
print("Data Shape:\n")
print(data.shape)
print("\n")

In [ ]:
# the columns 8 and 21 are of type objects: string
# the values of these columns will be checked, 
# looking for unexpected values that lead to having mixed data types

# column 8
print("Column 8: ")
print(cleaned_data['jet_1_phi'].unique())
# result: 
  # 1. float64 numeric values in string objects
  # 2. float64 numeric values in string objects and stored in a string object

# solution:
cleaned_data['jet_1_phi']=pd.to_numeric(cleaned_data['jet_1_phi'],errors='coerce')
print(cleaned_data['jet_1_phi'].unique().size)

# column 21
print("Column 21: ")
print(cleaned_data['jet_4_btag'].unique())
# result:
  # 1. float64 numeric values in string objects instead of float64 type
  # 2. alphabetical values in string objects

#solution:
cleaned_data['jet_4_btag']=pd.to_numeric(cleaned_data['jet_4_btag'], errors='coerce')
print(cleaned_data['jet_4_btag'].unique().size)


**Data Cleaning**

In [ ]:
# remove the training examples with NaN values from the dataset
cleaned_data.dropna(inplace=True)

For **regression and decision trees**, we will use the complete set of features (low-level and high-level combined) to take advantage of the manually constructed high-level features.

**Usage of the complete set of features**

**test_size** determines the proportion of the data that will be allocated for the testing set. In this case, test_size=0.2 means that **20%** of the data will be used **for testing**, and the remaining **80%** will be used for **training**.

**random_state** is an optional parameter that sets the random seed used by the random number generator. This **ensures** that the **random splitting of the data is reproducible**, meaning that if you run the same code multiple times with the same random_state value, you will get the same split of data into training and testing sets.

In [ ]:
# Split the dataset into training and testing sets
X = cleaned_data.iloc[:, 1:]  # Select all columns except the first one as features
y = cleaned_data.iloc[:, 0]   # Select the first column as the target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


**Linear Regression Model**

In [ ]:
# Create a linear regression model and fit it to the training set
model = LinearRegression()
model.fit(X_train, y_train)

# Use the model to make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model's mean squared error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error: {:.2f}'.format(mse))

**Logistic Regression**

In [ ]:
model = LogisticRegression(penalty= "l2", C=0.1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print('Accuracy:', accuracy_score(y_test, y_pred))

**Decision Trees**

In [ ]:
# Create a decision tree classifier and fit it to the training set
clf = DecisionTreeClassifier(random_state=42, max_depth=11, min_samples_split=11, splitter="best", min_impurity_decrease=0.0)
clf.fit(X_train, y_train)

# Evaluate the classifier's accuracy on the test set
accuracy_decisionTrees = clf.score(X_test, y_test)
print('Accuracy: {:.4f}'.format(accuracy_decisionTrees))

**XGBoost**

In [ ]:
# Define the XGB model
xgb_model = xgb.XGBClassifier()

# Train the model on the training data
xgb_model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = xgb_model.predict(X_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Model accuracy:", accuracy)

**Neural Network**

According to the paper, for the *Higgs boson benchmarks*, we can use a neural network to classify the data by training it on a set of input features and corresponding labels. For the Higgs boson benchmark, we can use *either* the **low-level features**, the **high-level features**, **or the complete set of features** (low-level and high-level combined) as **inputs** to the neural network.

The paper mentions that standard techniques in high-energy physics data analyses include **feed-forward neural networks with a single hidden layer**, which is an example of a *traditional shallow network*.

However, recent advances in deep-learning techniques may lift the limitations of shallow networks by automatically discovering powerful nonlinear feature combinations and providing better discrimination power than current classifiers.

To train a neural network, we need to choose its architecture (number of layers and units per layer), activation function, and other hyperparameters such as learning rate and regularization. The paper provides some details about the hyperparameters used for training deep neural networks on the Higgs boson. Once the network is trained, we can use it to make predictions on new data by feeding the input features through the network and computing the output probabilities.

According to the paper, for training deep neural networks on the Higgs boson, a **five-layer** neural network with **300 hidden units in each layer** was selected, with a **learning rate of 0.05**, and a weight decay coefficient of 1 × 10 −5.

Pre-training, extra hidden units, and additional hidden layers significantly increased training time without noticeably increasing performance.

Hidden units all used the **tanh activation function**. Weights were initialized from a normal distribution with zero mean and standard deviation 0.1 in the first layer, 0.001 in the output layer, and 0.05 in all other hidden layers. Gradient computations were made on mini-batches of size 100.

In [ ]:
# load the low-level features
X = cleaned_data.iloc[:, 2:23] # Select columns 2 to 22 as features: the 21 low-level features
y = cleaned_data.iloc[:, 0]    # Select the first column as the target variable


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Tanh**, short for hyperbolic tangent, is a commonly used activation function in neural networks. It is a nonlinear function that **maps the input values to a range between -1 and 1**. The tanh function is defined mathematically as:

tanh(x) = (exp(x) - exp(-x)) / (exp(x) + exp(-x))

Like other activation functions, tanh is used to introduce nonlinearity to the output of a neuron in a neural network. It is often used in the hidden layers of neural networks as it allows the network to **capture more complex patterns in the data**.

**Momentum** is a technique used in the *optimization of gradient descent* algorithms to *accelerate the convergence* of the training process.
**Momentum** adds a fraction of the previous update vector to the current update vector, which helps to smooth the optimization trajectory and avoid getting stuck in local minima.

In [ ]:
batch = 300
act = 'tanh'
neurons = 400
nb_epoch=1400
# Define the neural network architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(neurons, activation=act, input_dim=X_train.shape[1], kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.1)),
    tf.keras.layers.Dense(neurons, activation=act, kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05)),
    tf.keras.layers.Dense(neurons, activation=act, kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05)),
    tf.keras.layers.Dense(neurons, activation=act, kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05)),
    tf.keras.layers.Dense(neurons, activation=act,  kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05)),
    tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.001))
])

# Compile the model with specified hyperparameters
opt = tf.keras.optimizers.SGD(learning_rate=0.05, momentum=0.9)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

# create a callback to save the model with the highest accuracy
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')

# Train the model and evaluate on a validation set
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batch, epochs=nb_epoch, verbose=1, callbacks=[checkpoint])

# load the weights of the best model
model.load_weights('best_model.h5')

# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred = [1 if y >= 0.5 else 0 for y in y_pred]  # convert probabilities to binary predictions

# calculate the accuracy 
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: {:.4f}'.format(accuracy))

The decrease in the mean and standard deviation of the weight initialization is to prevent exploding gradients as we move forward towards the output layer during training.

**Regularizers and Stochastic Gradient Descent** (SGD) are two important techniques used in training neural networks to improve the accuracy and generalization performance of the model.

**Regularization** is a technique used to prevent overfitting by adding a penalty term to the loss function. Regularization encourages the model to have small weights and biases, which helps to reduce the complexity of the model and prevent it from fitting noise in the training data.
**L2 regularization** adds a penalty proportional to the square of the weights, and dropout randomly sets some neurons to zero during training. 

**SGD** is an optimization algorithm used to minimize the loss function during training. In contrast to batch gradient descent, which computes the gradient of the loss function using the entire dataset, SGD updates the model parameters using only a small subset of the data at a time. This makes it computationally efficient and able to handle large datasets

By **combining these two techniques**, the model is able to learn the underlying patterns in the training data while preventing it from overfitting and generalizing well to new data.
